In [ ]:
import h5py
import psana
import numpy as np
import time

# Specify the info of the experiment
lineName = 'amo'
experimentName = 'amok5415'
detInfo = 'pnccdFront'
run_num_list=['87']
file_num = len(run_num_list)
userName = 'haoyuan'
patternShape = (4,512,512)

for fileInd in range(file_num):
    
    print "for run number "+run_num_list[fileInd]
    t1=time.time()

    # Get run number
    run_num = run_num_list[fileInd]
    
    # Find the position of the cxi file
    fileName = '/reg/d/psdm/%s/%s/scratch/%s/psocake/r00%s/%s_00%s.cxi'%(lineName,
                                                                         experimentName,
                                                                         userName,
                                                                         run_num,
                                                                         experimentName,
                                                                         run_num)
    # load the data index file
    with h5py.File(fileName,mode='r+') as f:

        # Copy the event id
        event_id = np.copy(f['/LCLS/eventNumber'])
        
        # construct the detector
        ds = psana.DataSource('exp=%s:run=%s:idx'%(experimentName,run_num))
        run = ds.runs().next()
        times = run.times()
        env = ds.env()
        det = psana.Detector(detInfo, env)
        
        # Get pattern number
        pattern_num = float(len(event_id))

        pattern_mean = np.zeros(patternShape)
        pattern_sum = np.zeros(patternShape)
        
        for num in range(len(event_id)):
             
            # Get the image and mask
            evt = run.event(times[int(event_id[num])])
            pattern = det.calib(evt)
            
            # Get the average and sum
            pattern_sum += pattern
            pattern_mean += pattern/pattern_num

            
    t2 = time.time()
    print ('total time usage: %f'%(t2-t1))
    
    ## Save the result
    name = '/reg/d/psdm/%s/%s/scratch/%s/psocake/r00%s/%s_mean.npy'%(lineName,
                                                                        experimentName,
                                                                        userName,
                                                                        run_num,
                                                                        userName)
    np.save(name, pattern_mean)
    
    name = '/reg/d/psdm/%s/%s/scratch/%s/psocake/r00%s/%s_sum.npy'%(lineName,
                                                                        experimentName,
                                                                        userName,
                                                                        run_num,
                                                                        userName)   
    np.save(name, pattern_sum)